In [2]:
import numpy as np
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train.reshape(x_train.shape[0], -1).T.astype('f4')
x_test = x_test.reshape(x_test.shape[0], -1).T.astype('f4')

y_train, y_test = y_train.astype(np.int32), y_test.astype(np.int32)
#train on subsets
x_train = x_train[:,:10000]
y_train = y_train[:10000]


In [3]:
x_train.shape

(784, 10000)

In [40]:
def init_params():
    w1 = np.random.randn(10, 28*28).astype('f4') * np.sqrt(1. / (28*28))
    b1 = np.random.randn(10, 1).astype('f4') * 0.01
    w2 = np.random.randn(10, 10).astype('f4') * np.sqrt(1. / 10)
    b2 = np.random.randn(10, 1).astype('f4') * 0.01
    return w1, b1, w2, b2
def ReLu(z):
    return np.maximum(z, 0)

def ReLu_deriv(z):
    return (z > 0).astype(float)

def Softmax(z):
    exp_z = np.exp(z - np.max(z, axis=0, keepdims=True))
    return exp_z / np.sum(exp_z, axis=0, keepdims=True)

def forward_prop(w1, b1, w2, b2, X):
    z1 = np.dot(w1, X) + b1
    a1 = ReLu(z1)
    z2 = np.dot(w2, a1) + b2
    a2 = Softmax(z2)
    return z1, a1, z2, a2

def one_hot(y, num_classes=10):
    y = y.reshape(-1)
    one_hot = np.zeros((num_classes, y.size))
    one_hot[y, np.arange(y.size)] = 1
    return one_hot

def back_prop(z1, a1, z2, a2, w2, x, y):
    m = y.size
    one_hot_y = one_hot(y, num_classes=10)

    dz2 = a2 - one_hot_y
    dw2 = (1 / m) * np.dot(dz2, a1.T)
    db2 = (1 / m) * np.sum(dz2, axis=1, keepdims=True)

    dz1 = np.dot(w2.T, dz2) * ReLu_deriv(z1)
    dw1 = (1 / m) * np.dot(dz1, x.T)
    db1 = (1 / m) * np.sum(dz1, axis=1, keepdims=True)

    return dw1, db1, dw2, db2

def update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha):
    w1 -= alpha * dw1
    b1 -= alpha * db1
    w2 -= alpha * dw2
    b2 -= alpha * db2
    return w1, b1, w2, b2

def get_predictions(A2):
    return np.argmax(A2, axis=0)

def get_accuracy(predictions, y):
    return np.mean(predictions == y)



In [5]:
def gradient_descent(X, Y, iterations, alpha):
    w1, b1, w2, b2 = init_params()

    for i in range(iterations):
        z1, a1, z2, a2 = forward_prop(w1, b1, w2, b2, X)
        dw1, db1, dw2, db2 = back_prop(z1, a1, z2, a2, w2, X, Y)
        w1, b1, w2, b2 = update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha)

        if i % 50 == 0:
            accuracy = get_accuracy(get_predictions(a2), Y)
            print(f"Iteration {i}: Accuracy = {accuracy:.4f}")
    return w1, b1, w2, b2

w1, b1, w2, b2 = gradient_descent(x_train, y_train, iterations=1000, alpha=0.1)


Iteration 0: Accuracy = 0.0790
Iteration 50: Accuracy = 0.7758
Iteration 100: Accuracy = 0.8456
Iteration 150: Accuracy = 0.8731
Iteration 200: Accuracy = 0.8890
Iteration 250: Accuracy = 0.8997
Iteration 300: Accuracy = 0.9066
Iteration 350: Accuracy = 0.9099
Iteration 400: Accuracy = 0.9118
Iteration 450: Accuracy = 0.9141
Iteration 500: Accuracy = 0.9179
Iteration 550: Accuracy = 0.9201
Iteration 600: Accuracy = 0.9213
Iteration 650: Accuracy = 0.9229
Iteration 700: Accuracy = 0.9249
Iteration 750: Accuracy = 0.9274
Iteration 800: Accuracy = 0.9290
Iteration 850: Accuracy = 0.9309
Iteration 900: Accuracy = 0.9317
Iteration 950: Accuracy = 0.9327


In [41]:
#test data
def test_predictions(w1, b1, w2, b2,x,y):
  _, _, _, a2 = forward_prop(w1, b1, w2, b2, x)
  accuracy = get_accuracy(get_predictions(a2), y)
  print(f"Accuracy = {accuracy:.4f}")
def predict_single(w1, b1, w2, b2,x):
  _, a1, _, a2 = forward_prop(w1, b1, w2, b2, x)
  return a1,a2,get_predictions(a2)

test_predictions(w1, b1, w2, b2,x_test,y_test)

Accuracy = 0.9102


Now we will start converting the nural network to work on the GPU using compute shaders we will start first by the forward pass 
we initialize our modernGL context and set some variables.
N (Batch Size): Number of input samples processed at once.
d (Input Dimension): Size of each input vector (784 for MNIST images of 28×28).
h (Hidden Layer Size): Number of neurons in the hidden layer.
num_classes (Output Size): Number of output classes (10 for MNIST digits 0-9).

In [42]:
import moderngl
import numpy as np

# Initialize ModernGL context
ctx = moderngl.create_standalone_context()

# Network dimensions
INPUT_SIZE = 784
HIDDEN_SIZE = 10
OUTPUT_SIZE = 10

In [43]:
DATA_POINT = 10


#we transpose it becaue GPU require column-major order for matrices
w1_buffer = ctx.buffer(w1.T.flatten().astype(np.float32).tobytes())
b1_buffer = ctx.buffer(b1.flatten().astype(np.float32).tobytes())
w2_buffer = ctx.buffer(w2.T.flatten().astype(np.float32).tobytes())
b2_buffer = ctx.buffer(b2.flatten().astype(np.float32).tobytes())
output_buffer = ctx.buffer(reserve=OUTPUT_SIZE * 4)

a1,a2,pred = predict_single(w1, b1, w2, b2,x_train[:,DATA_POINT:DATA_POINT+1])
print(y_train[DATA_POINT])
print(pred)

3
[3]


In [44]:
program = ctx.compute_shader(open("ForwardPass.glsl").read())

w1_buffer.bind_to_storage_buffer(1)
b1_buffer.bind_to_storage_buffer(2)
w2_buffer.bind_to_storage_buffer(3)
b2_buffer.bind_to_storage_buffer(4)
output_buffer.bind_to_storage_buffer(5)

input_buffer = ctx.buffer(x_train[:,DATA_POINT].astype(np.float32).tobytes())
input_buffer.bind_to_storage_buffer(0)


In [50]:


def predict_single_GPU():
    # Run shader
    program.run(1)
    results = np.frombuffer(output_buffer.read(), dtype=np.float32)
    # Get results
    return np.argmax(results)

%timeit predict_single_GPU()
%timeit predict_single(w1, b1, w2, b2,x_train[:,DATA_POINT:DATA_POINT+1])


153 μs ± 5.47 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
26.4 μs ± 897 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


if we run the timeit benchmark for predicting a single data point in both the CPU and The GPU we found that the CPU is around 6 times faster than the GPU, CPU ~ 26.4 | GPU ~ 153. disapointing :(

but if we take a closer look and separate the 2 main functions that are called in the GPU predict function we will find that each one of them take around 4 to 7 μs and 4+7 μs << 153 μs


In [53]:
%timeit program.run(1)
%timeit np.frombuffer(output_buffer.read(), dtype=np.float32)

5.76 μs ± 2.17 μs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
The slowest run took 10.86 times longer than the fastest. This could mean that an intermediate result is being cached.
3.51 μs ± 4.78 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)



this means that most of the time the CPU was waiting for the data to be transfered from the GPU to the CPU which is a slow process. so if we manage to minimize the numbers of GPU to CPU transfers we can boost the preformance significantly we cann't do that predicting single data point but we can do it when we dealing with large chunks or whole datasets using batches which GPU are really good at processing batches